<a href="https://colab.research.google.com/github/TrinhBaThinh-257275/AI-Sign-Language/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import các thư viện cần thiết 
import os
import warnings
import cv2
import keras
import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
from PIL import Image
from keras import models, layers, optimizers
from keras.applications import VGG16
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# truy cập vào mydrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# truy cập vào thư mục trong Colab Notebooks
%cd /content/drive/MyDrive/Colab Notebooks/Project

/content/drive/MyDrive/Colab Notebooks/Project


In [4]:
# Cài các thư viện
!pip install opencv-python
!pip install keras
!pip install tensorflow
!pip install matplotlib
!pip install pandas
!pip install pillow
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=63edf9430761fbf9c56e29304f5ea5972224a6e1b8a8e266fedeba402d378775
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [5]:
# Định Nghĩa các biến từ tên đầu các ảnh
gestures = {'A_': 'A',
            'B_': 'B',
            'C_': 'C',
            'D_': 'D',
            'E_': 'E',
            'F_': 'F',
            'G_': 'G',
            'I_': 'I',
            'L_': 'L',
            'O_': 'O',
            'R_': 'R',
            'U_': 'U',
            'V_': 'V',
            'X_': 'X'}
# Biến ký tự về số và số về ký tự
gestures_map = {'A': 0,
                'B': 1,
                'C': 2,
                'D': 3,
                'E': 4,
                'F': 5,
                'G': 6,
                'I': 7,
                'L': 8,
                'O': 9,
                'R': 10,
                'U': 11,
                'V': 12,
                'X': 13}
gesture_names ={0 :'A',
                1 :'B',
                2 :'C',
                3 :'D',
                4 :'E',
                5 :'F',
                6 :'G',
                7 :'I',
                8 :'L',
                9 :'O',
                10:'R',
                11:'U',
                12:'V',
                13:'X'}
# Đường dẫn lưu ảnh trên và model 
image_path = '/content/drive/MyDrive/Colab Notebooks/Project/training'
models_path = '/content/drive/MyDrive/Colab Notebooks/Project/model/saved_model.hdf5'
rgb = False
imageSize = 100 #Kích thước ảnh

# Hàm xử lý ảnh resize về 100x100 và chuyển về numpy array
def process_image(path):
    img = Image.open(path)
    img = img.resize((imageSize, imageSize))
    img = np.array(img)
    return img

# Xử lý dữ liệu đầu vào
def process_data(X_data, y_data):
    X_data = np.array(X_data, dtype = 'float32')
    if rgb:
        pass
    else:
        X_data = np.stack((X_data,)*1, axis=-1) # Nếu ảnh đen trắng thì sẽ nhân 1 stack lên 
    X_data /= 255
    y_data = np.array(y_data)
    y_data = to_categorical(y_data)
    return X_data, y_data

# Hàm duyệt thư mục ảnh để train
def walk_file_tree(image_path):
    X_data = [] 
    y_data = []
    for directory, subdirectories, files in os.walk(image_path):
        for file in files:
            if not file.startswith('.'):
                path = os.path.join(directory, file)
                gesture_name = gestures[file[0:2]]
                print(gesture_name)
                print(gestures_map[gesture_name])
                y_data.append(gestures_map[gesture_name])
                X_data.append(process_image(path))
            else:
                continue
    X_data, y_data = process_data(X_data, y_data)
    return X_data, y_data

# Load data X vs Y và lưa vào X_data và Y_data
X_data, y_data = walk_file_tree(image_path)

# Phân chia data train vs test theo tỷ lệ 80/20
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state=12, stratify=y_data)

# Đặt các checkpoint để lưu lại model tốt nhất
model_checkpoint = ModelCheckpoint(filepath=models_path, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_acc',
                               min_delta=0,
                               patience=10,
                               verbose=1,
                               mode='auto',
                               restore_best_weights=True)

# Khởi tạo model
model1 = VGG16(weights='imagenet', include_top=False, input_shape=(imageSize, imageSize, 3))
optimizer1 = optimizers.Adam()
base_model = model1

# Thêm các lớp bên trên
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
x = Dense(128, activation='relu', name='fc2a')(x)
x = Dense(128, activation='relu', name='fc3')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu', name='fc4')(x)

predictions = Dense(14, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Đóng các lớp base_model, chỉ train lớp trên mình đã thêm vào
for layer in base_model.layers:
    layer.trainable = False
# Hàm train 50 lần 
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stopping, model_checkpoint])

Streaming output truncated to the last 5000 lines.
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
R
10
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
D
3
E
4
E
4
E
4
E
4
E
4
E
4
E
4
E
4
E
4
E
4
E
4
E
4
E
4
E
4
E
4
E
4
E
4

28/28 [==============================] - 20s 375ms/step - loss: 1.7367 - accuracy: 0.4716 - val_loss: 0.2578 - val_accuracy: 0.9679
Epoch 2/50
27/28 [===========================>..] - ETA: 0s - loss: 0.2613 - accuracy: 0.9303

28/28 [==============================] - 5s 170ms/step - loss: 0.2595 - accuracy: 0.9309 - val_loss: 0.0191 - val_accuracy: 0.9931
Epoch 3/50
27/28 [===========================>..] - ETA: 0s - loss: 0.0729 - accuracy: 0.9829

28/28 [==============================] - 5s 169ms/step - loss: 0.0728 - accuracy: 0.9831 - val_loss: 0.0037 - val_accuracy: 0.9989
Epoch 4/50
27/28 [===========================>..] - ETA: 0s - loss: 0.0321 - accuracy: 0.9936

28/28 [==============================] - 5s 170ms/step - loss: 0.0328 - accuracy: 0.9934 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 5/50
27/28 [===========================>..] - ETA: 0s - loss: 0.0268 - accuracy: 0.9933

...

28/28 [==============================] - 5s 170ms/step - loss: 2.8059e-04 - accuracy: 1.0000 - val_loss: 3.3340e-06 - val_accuracy: 1.0000
Epoch 47/50
27/28 [===========================>..] - ETA: 0s - loss: 1.8550e-04 - accuracy: 1.0000

28/28 [==============================] - 5s 170ms/step - loss: 1.8400e-04 - accuracy: 1.0000 - val_loss: 6.1381e-08 - val_accuracy: 1.0000
Epoch 48/50
27/28 [===========================>..] - ETA: 0s - loss: 1.4559e-04 - accuracy: 1.0000

28/28 [==============================] - 5s 171ms/step - loss: 1.4444e-04 - accuracy: 1.0000 - val_loss: 9.4464e-08 - val_accuracy: 1.0000
Epoch 49/50
27/28 [===========================>..] - ETA: 0s - loss: 5.7540e-05 - accuracy: 1.0000

28/28 [==============================] - 5s 171ms/step - loss: 5.7045e-05 - accuracy: 1.0000 - val_loss: 4.5524e-08 - val_accuracy: 1.0000
Epoch 50/50
27/28 [===========================>..] - ETA: 0s - loss: 3.5384e-04 - accuracy: 1.0000

28/28 [==============================] - 6s 202ms/step - loss: 3.5082e-04 - accuracy: 1.0000 - val_loss: 1.0253e-08 - val_accuracy: 1.0000


In [6]:
# Lưu model dã train ra file
model.save('train.h5')